# Classification Models to Identify Water Pump Functionality in Tanzania

# Overview

This project consists of classification models to identify water pump functionality in Tanzania. Limited access to safe water is a major health risk. These models are potential tools for government agencies or non-governmental organizations to identify areas with limited access to clean water. Through an iterative modeling process, we produced a model that predicted pump functionality with 85% accuracy. By identifying non-functional water pumps, organizations can divert resources to areas in need of assistance and improve water access and health in Tanzania.

# Business Understanding

Millions of people in Tanzania lack access to safe water. This results in paying high prices for water from vendors or collecting water from unsafe natural sources. In order to combat this problem, resources must be allocated to fix non-functioning water distribution points.
Age is an important metric in predicting the condition of distribution points. Older pumps and engine systems are more likely to fail than newer ones. In many cases age related data is not available.
The goal of this analysis is to build the model that can predict the condition of waterpoints based on their other features such as regional factors, installer, type of pump, population and others

# Data Understanding

The data was sourced from the Taarifa waterpoint dashboard, which aggregates data from the Tanzania Ministry of Water. The information collected was recorded by GeoData Consultants Ltd. There are 59,400 rows and 40 columns in the "water_well_train_data.csv". 

Our target data is stored in "water_well_train_labels.csv". There are 59,400 rows and 2 columns in this csv file. The two columns in this csv file are 'id' and 'status_group'. The 'id' column aligns with the 'id' column in the "water_well_train_data.csv" file. Our target column is 'status_group' which consists of three values describing the status of a water pump: "functional", "functional needs repair", and "non-functional".

# Data Cleaning and EDA

In [1]:
#import statements
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as ply
import seaborn as sns

#sci-kit learn
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
#import train data
#DO NOT LOOK AT TEST DATA UNTIL VALIDATION
df_train = pd.read_csv('./data/water_well_train_data.csv')

In [3]:
df_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
# info of train data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

The data contains 59,400 rows and 40 columns.

Our target column is stored in a separate csv file. 

In [5]:
#import target information
df_label = pd.read_csv('./data/water_well_train_labels.csv')

In [6]:
df_label.shape

(59400, 2)

In [7]:
df_label.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [8]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [9]:
df_label['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [10]:
df_label['status_group'].value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

There are three target classifications: functional (54%), non-functional (38%), and function needs repair (7%). We combined the 'status_group' dataframe with the train_data dataframe.

In [11]:
#combine train and label dataframes prior to cleaning to address any dropped rows
df = df_train.join(other = df_label, rsuffix = '_label')

In [12]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


A quick confirmation to see that the columns in the dataframe are properly aligned. The id values from each dataframe should match.

In [13]:
#check that id columns align
df[df['id'] != df['id_label']]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


Now that our features and target were in the same dataframe, we could begin exploring and cleaning the data.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

We have 42 columns: the majority of them were object type with a few numerical.

Columns with nulls:
- funder
- installer
- subvillage
- public_meeting
- scheme_management
- scheme_name
- permit

In [15]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,id_label
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475,37115.131768
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547,21453.128371
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000,18519.750000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,37061.500000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,55656.500000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,74247.000000


## Dealing with Nulls

We looked at every column with null values to determine how to address those missing values.

### funder

In [16]:
#who funded the well
print(df['funder'].isna().sum())
df['funder'].value_counts()

3635


Government Of Tanzania        9084
Danida                        3114
Hesawa                        2202
Rwssp                         1374
World Bank                    1349
                              ... 
Pentekoste                       1
Usambala Sister                  1
Rumaki                           1
Friedkin Conservation Fund       1
Rashid Seng'ombe                 1
Name: funder, Length: 1897, dtype: int64

There were 3,635 nulls in 'funder'.

In [17]:
unique_funder = list(df['funder'].unique())
print(len(unique_funder))
unique_funder

1898


['Roman',
 'Grumeti',
 'Lottery Club',
 'Unicef',
 'Action In A',
 'Mkinga Distric Coun',
 'Dwsp',
 'Rwssp',
 'Wateraid',
 'Isingiro Ho',
 'Private',
 'Danida',
 'World Vision',
 'Lawatefuka Water Supply',
 'Biore',
 'Rudep',
 'Hesawa',
 'Twe',
 'Isf',
 'African Development Bank',
 'Government Of Tanzania',
 'Sobodo',
 'Water',
 'Private Individual',
 'Undp',
 nan,
 'Not Known',
 'Kirde',
 'Cefa',
 'Ces(gmbh)',
 'European Union',
 'Lga',
 'District Council',
 'Muwsa',
 'Dwe/norad',
 'Kkkt_makwale',
 'Sawaka',
 'Ces (gmbh)',
 'Olgilai Village Community',
 'Kkkt',
 'Roman Catholic',
 'Norad',
 'Adra',
 'Sema',
 'Piusi',
 'Dwe',
 'Rc Church',
 'Swisland/ Mount Meru Flowers',
 'Ifad',
 'Swedish',
 'Idc',
 'He',
 'Isf/tacare',
 'Jica',
 'Mzee Sh',
 'Aict',
 'Tcrs',
 'Kiuma',
 'Germany Republi',
 'Netherlands',
 'Ruthe',
 'Tulawaka Gold Mine',
 'Nethalan',
 'Tasaf',
 'Concern World Wide',
 'Wfp',
 'Lips',
 'Sida',
 'World Bank',
 'Tanza',
 '0',
 'Sw',
 'Shipo',
 'Fini Water',
 'Kanisa',
 'Ox

In [18]:
df[df['funder'] == '0']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
145,28669,50.0,2013-03-21,0,12,0,39.185710,-6.892593,Chekanao,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,28669,functional
152,60983,0.0,2013-03-16,0,-15,0,39.527114,-6.988748,Msikitini,0,...,salty,dry,dry,machine dbh,borehole,groundwater,other,other,60983,non functional
393,39749,0.0,2013-03-18,0,28,0,39.159887,-6.902548,Kwa Chambuso,0,...,unknown,unknown,unknown,machine dbh,borehole,groundwater,other,other,39749,non functional
417,15832,50.0,2013-03-22,0,30,0,39.178404,-6.938013,Ccm Kivule,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,15832,non functional
428,50233,0.0,2013-03-12,0,30,0,39.178849,-6.973206,Ofisi Ya Kata,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,other,other,50233,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59237,2138,59.0,2013-03-19,0,81,0,39.119109,-6.898919,Kata,0,...,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,2138,non functional
59243,3396,50.0,2013-03-16,0,-20,0,39.524021,-6.984802,Kwa Mariwala,0,...,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,3396,functional
59276,62818,50.0,2013-03-21,0,18,0,39.183790,-6.897566,Kwa Mkunduge,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,62818,functional
59351,55322,50.0,2013-03-18,0,-19,0,39.534599,-7.088183,Kwa China,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,55322,functional


We saw in the value_counts that there was already a 'Not Known' value so the nulls were changed to 'Not Known'.

In [19]:
#replace nulls in 'funder' col with "Not Known"
df['funder'].fillna(value="Not Known", inplace=True)

A value of '0' was also treated a a null. These values seemed out-of-place in the column and didn't seem to describe a particular organization or individual responsible for funding.

In [20]:
df['funder'] = df['funder'].replace(to_replace={'0':'Not Known'}, value=None)

In [21]:
#confirm replacement of nulls
df['funder'].isna().sum()

0

In [22]:
df[df['funder'] == '0']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


We've confirmed removal of null values and '0' values from the 'funder' column.

### installer

In [23]:
#organization that installed the well
df['installer'].value_counts()

DWE                  17402
Government            1825
RWE                   1206
Commu                 1060
DANIDA                1050
                     ...  
Ta                       1
Zao                      1
Marumbo Community        1
BATIST CHURCH            1
Regina group             1
Name: installer, Length: 2145, dtype: int64

Installer had 2,154 unique values which could be an issue if we planned on including this as a categorical feature in our modeling.

In [24]:
df['installer'].isna().sum()

3655

There were 3,655 nulls in 'installer'. This value matched the number of nulls we saw in 'funder', which could suggest that the same rows had missing values.

In [25]:
list(df['installer'].unique())

['Roman',
 'GRUMETI',
 'World vision',
 'UNICEF',
 'Artisan',
 'DWE',
 'DWSP',
 'Water Aid',
 'Private',
 'DANIDA',
 'Lawatefuka water sup',
 'WEDECO',
 'Danid',
 'TWE',
 'ISF',
 'Kilolo Star',
 'District council',
 'Water',
 'WU',
 nan,
 'Not known',
 'Central government',
 'CEFA',
 'Commu',
 'Accra',
 'World Vision',
 'LGA',
 'MUWSA',
 'KKKT _ Konde and DWE',
 'Government',
 'Olgilai village community',
 'KKKT',
 'RWE',
 'Adra /Community',
 'SEMA',
 'SHIPO',
 'HESAWA',
 'ACRA',
 'Community',
 'IFAD',
 'Sengerema Water Department',
 'HE',
 'ISF and TACARE',
 'Kokeni',
 'DA',
 'Adra',
 'ALLYS',
 'AICT',
 'KIUMA',
 'CES',
 'District Counci',
 'Ruthe',
 'Adra/Community',
 'Tulawaka Gold Mine',
 'KKT C',
 'Hesawa',
 'Water board',
 'LOCAL CONTRACT',
 'WFP',
 'LIPS',
 'TASAF',
 'World',
 '0',
 'SW',
 'Shipo',
 'Fini water',
 'Kanisa',
 'OXFARM',
 'VILLAGE COUNCIL Orpha',
 'Villagers',
 'Idara ya maji',
 'FPCT',
 'WVT',
 'Ir',
 'DANID',
 'Angli',
 'secondary school',
 'Amref',
 'JBG',
 'DAD

Again, we saw there was a 'Not known' value in 'installer', which we decided to use to replace nulls.

In [26]:
df['installer'].fillna(value='Not known', inplace=True)

In [27]:
df['installer'].isna().sum()

0

We noticed significant overlap in values for 'funder' and 'installer'. From our business understanding, this made sense as organizations who were funding water pumps were also likely to be responsible for their installation.

In [28]:
#check cases where the installer is not also the funder
df[df['funder'] != df['installer']]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,9944,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,60739,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,27263,functional
59397,37057,0.0,2011-04-11,Not Known,0,Not known,34.017087,-8.750434,Mashine,0,...,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,37057,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,31282,functional


In [29]:
#investiating funder and installer relationship with World Bank as a specific value case
df[df['installer']=='World Bank']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
280,9474,250.0,2013-02-12,World Bank,1343,World Bank,30.017098,-4.332583,Jeshini,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,9474,functional
304,43345,0.0,2012-10-23,World Bank,0,World Bank,33.430917,-4.389084,Shule Ya Msingi,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,43345,functional
2071,28588,0.0,2012-10-23,World Bank,0,World Bank,33.436073,-4.421944,Mwanza Road,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,28588,functional
5699,74077,0.0,2012-10-18,World Bank,0,World Bank,33.435268,-4.671744,Kwa Fupe,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,74077,non functional
6327,44441,0.0,2012-10-12,World Bank,0,World Bank,33.150261,-3.705625,Kwa Maraba,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,44441,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58348,54574,0.0,2012-10-23,World Bank,0,World Bank,33.442457,-4.426592,Kilabuni,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,54574,functional
58434,54545,250.0,2013-02-12,World Bank,1311,World Bank,30.017351,-4.323850,Mission,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,54545,functional
58681,6335,250.0,2013-02-12,World Bank,1306,World Bank,30.017716,-4.308701,Kwa Thomas,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,6335,functional
58691,67819,0.0,2012-10-12,World Bank,0,World Bank,33.142605,-3.699442,Kwa Nyamizi Maswa,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67819,non functional


In [30]:
df[(df['installer']=='World') & (df['funder']!='World Bank')]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
33726,63910,0.0,2011-04-02,Nethe,141,World,38.204463,-6.870355,Kwa Kiwele,0,...,salty,insufficient,insufficient,shallow well,shallow well,groundwater,other,other,63910,non functional


In [31]:
df[df['installer'] == 'Not known']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
34,41583,0.0,2011-02-23,Not Known,-41,Not known,39.812912,-7.889986,Msikitini Wa Ijumaa,0,...,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,41583,functional
35,57355,0.0,2013-03-28,Not Known,1546,Not known,36.618699,-3.293003,Sekondari,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,57355,non functional
43,19282,0.0,2013-01-15,Not Known,1642,Not known,34.967789,-4.628921,Mvae Primary,0,...,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,19282,non functional
47,13620,0.0,2011-07-27,Not Known,0,Not known,33.540607,-9.172905,Mahakamani,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,13620,functional
65,51072,0.0,2013-02-09,Not Known,1415,Not known,34.621598,-5.173136,Nyambi,0,...,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,51072,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59357,46563,0.0,2013-02-19,Not Known,1635,Not known,34.971841,-5.098362,Shabani,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,46563,non functional
59366,55232,0.0,2013-02-02,Not Known,1541,Not known,34.765729,-5.027725,Joshoni,0,...,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,55232,functional
59370,14796,200.0,2013-01-29,Not Known,1154,Not known,30.058731,-4.902633,Village Office,0,...,unknown,unknown,unknown,unknown,other,unknown,other,other,14796,functional
59376,34716,0.0,2013-02-03,Not Known,1581,Not known,34.821039,-5.076258,Nasingo,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,other,other,34716,non functional


In [32]:
df['installer'].value_counts(normalize=True)[:20]

DWE                   0.292963
Not known             0.061818
Government            0.030724
RWE                   0.020303
Commu                 0.017845
DANIDA                0.017677
KKKT                  0.015118
Hesawa                0.014141
0                     0.013081
TCRS                  0.011902
Central government    0.010471
CES                   0.010269
Community             0.009310
DANID                 0.009293
District Council      0.009276
HESAWA                0.009074
LGA                   0.006869
World vision          0.006869
WEDECO                0.006684
TASAF                 0.006667
Name: installer, dtype: float64

As with the 'funder' column, we treated values of '0' as nulls and replaced them with 'Not known'.

In [33]:
df['installer'] = df['installer'].replace(to_replace={'0':'Not known'}, value=None)

In [34]:
df['installer'].value_counts(normalize=True)[:20]

DWE                   0.292963
Not known             0.074899
Government            0.030724
RWE                   0.020303
Commu                 0.017845
DANIDA                0.017677
KKKT                  0.015118
Hesawa                0.014141
TCRS                  0.011902
Central government    0.010471
CES                   0.010269
Community             0.009310
DANID                 0.009293
District Council      0.009276
HESAWA                0.009074
World vision          0.006869
LGA                   0.006869
WEDECO                0.006684
TASAF                 0.006667
District council      0.006599
Name: installer, dtype: float64

In [35]:
df[df['installer'] == "0"]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


### subvillage

In [36]:
#geographic location
df['subvillage'].isna().sum()

371

In [37]:
df['subvillage'].value_counts()

Madukani        508
Shuleni         506
Majengo         502
Kati            373
Mtakuja         262
               ... 
Masaladi          1
Mgodi             1
Ipwasi            1
Tobo              1
Bombambili 2      1
Name: subvillage, Length: 19287, dtype: int64

There were 3,635 nulls in 'subvillage'. This column has over 19,287 unique values.

In [38]:
list(df['subvillage'].unique())

['Mnyusi B',
 'Nyamara',
 'Majengo',
 'Mahakamani',
 'Kyanyamisa',
 'Moa/Mwereme',
 'Ishinabulandi',
 'Nyawishi Center',
 'Imalauduki',
 'Mkonomre',
 'Mizugo',
 'Ngondombwito',
 'Nkilifa',
 'Omarini',
 'Mwabasabi',
 'Tunzi',
 'Kidudumo',
 'Yeriko',
 'Center',
 'Manyanya',
 'Ibabachegu',
 'Mkanivega',
 'Mkonga Juu',
 'Msasa',
 'Kitereni',
 'Shuleni',
 'Chakahaya',
 'Kiyao',
 'Merali',
 'Karume',
 'Kudipera',
 'Mosheni',
 'Lupanga A',
 'Kilombero B',
 'Afya',
 'Ndanganyika',
 'Baura',
 'Mwanzala',
 'Nyabwai B',
 'Reli B',
 'Kilunduwe',
 'Sokoni',
 'Mwarufyu',
 'Marurani Juu',
 'Isenegeja',
 'Kachulu',
 'Mpandapanda',
 'Mlandege Juu',
 'Ikanga',
 'Msaranga Street',
 'Maporomoko',
 'Isimba',
 'Kagoye B',
 'Dhobi Street',
 'Msufini',
 'Soyekiutu',
 'Ants B',
 "Izimbya 'A'",
 'Nairobi',
 'Nkaloi',
 'Kiganza Centre',
 'Ulkusare',
 'Mzimba',
 'Mfumbu',
 'Mahalule',
 'Kihanga',
 'Iponda',
 'Kisoro',
 'Mtakuja',
 'Lembuka',
 'Mapinduzi',
 'Kalimungoma',
 'Umkituri',
 'Usita',
 'Komoro',
 'Kulasi

In [39]:
df[df['subvillage'] == 'Unknown']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


Unlike our previous cases, there wasn't a clear value to change nulls to. We decided to create a new 'Unknown' value to replace nulls rather than dropping these rows. 

In [40]:
df['subvillage'].fillna(value='Unknown', inplace=True)

In [41]:
df['subvillage'].isna().sum()

0

### public_meeting

In [42]:
df['public_meeting'].isna().sum()

3334

In [43]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

There were 3,334 nulls in 'public_meeting'. Given the large imbalance between True and False value counts, we decided that nulls should match the majority class, True.

In [44]:
df['public_meeting'].fillna(value=True, inplace=True)

In [45]:
df['public_meeting'].isna().sum()

0

### scheme_management

In [46]:
#who operates the waterpoint (organization/category)
df['scheme_management'].isna().sum()

3877

There were 3,877 nulls in 'scheme_management'.

In [47]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [48]:
#investigating the one value_count of None
df[df['scheme_management'] == "None"]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
23603,23849,50.0,2013-03-18,Not Known,-11,Not known,39.431194,-7.100783,Kwa Nyamtawa,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,23849,functional


In [49]:
df[df['funder'] == '0']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


Since there was already an instance of 'None' in this column, we changed all nulls to 'None'.

In [50]:
df['scheme_management'].fillna(value='None', inplace=True)

In [51]:
df['scheme_management'].isna().sum()

0

### scheme_name

This column had a large number of nulls: 28,166.
Since the values appeared to be individual names with great diversity, we decided this column was unusuable for our modeling. However, we didn't drop it from the dataframe since our modeling process uses a pipeline for this purpose.

In [52]:
#who operates the waterpoint
#how is this information different from scheme_management?
df['scheme_name'].value_counts()

K                      682
None                   644
Borehole               546
Chalinze wate          405
M                      400
                      ... 
Nkwe                     1
BUWADA                   1
BL Bonifas Kingu         1
BL Losei                 1
DMK:Anglican church      1
Name: scheme_name, Length: 2696, dtype: int64

In [53]:
df['scheme_name'].isna().sum()

28166

In [54]:
unique_scheme = list(df['scheme_name'].unique())
print(len(unique_scheme))
unique_scheme

2697


['Roman',
 nan,
 'Nyumba ya mungu pipe scheme',
 'Zingibali',
 'BL Bondeni',
 'None',
 "wanging'ombe water supply s",
 'Makanj',
 'Kidabu',
 'Mashangwi',
 'Quick wins Program',
 'Komaka mandaka',
 'Sobodo Borehole Scheme',
 'Kitukuni water supply',
 'BL Mwakikoti',
 'Chalinze wate',
 'Mae pipeline',
 'UNDP',
 'Ngana water supplied scheme',
 'Itun',
 'Bomala',
 'Kirua kahe pumping water trust',
 'Misiwa',
 'Mtwango water supplied sche',
 'K',
 'Machumba estate pipe line',
 'SHIMASA',
 'wangama water supply scheme',
 'Shirimatunda Water Supply',
 'Laela group water Supp',
 'Makwale water supplied sche',
 "Nyang'hwale",
 'Njoro Water Supply',
 'Kirua kahe gravity water supply trust',
 'Olgilai pipe line',
 'Mabula mountains spr',
 'Mkongoro One',
 'Maambreni gravity water supply',
 'Mwando water supply',
 'M',
 'Kaisho/Isingiro w',
 "wanging'ombe supply scheme",
 'Tove Mtwango gravity Scheme',
 'Tengeru gravity water supply',
 'Kulasi water supply',
 'Malemb',
 'Tawa',
 'Loruvani gravity 

We replaced the nulls with 'None'. Although there is a strong case to drop this column (which now contains 28,810 'None' values), we left it in the dataframe.

In [55]:
df['scheme_name'].fillna(value='None', inplace = True)

In [56]:
df['scheme_name'].isna().sum()

0

In [57]:
df['scheme_name'].value_counts()

None                   28810
K                        682
Borehole                 546
Chalinze wate            405
M                        400
                       ...  
Nkwe                       1
BUWADA                     1
BL Bonifas Kingu           1
BL Losei                   1
DMK:Anglican church        1
Name: scheme_name, Length: 2696, dtype: int64

### Permit

In [58]:
df['permit'].isna().sum()

3056

There were 3,056 nulls in 'permit'.

We assumed that if a permit status is unknown, there is no permit. The nulls were changed to False.

In [59]:
#if the waterpoint is permitted
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [60]:
df['permit'].isna().sum()

3056

In [61]:
df['permit'].fillna(value=False, inplace=True)

In [62]:
df['permit'].isna().sum()

0

## Column Exploration

Now that null values have been treated, we continued our exploration of the data by examining columns individually.

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59400 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [64]:
#total static head (amount of water available to waterpoint)
df['amount_tsh'].value_counts()

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

In [65]:
df[df['amount_tsh'] == 0.0]['status_group'].value_counts()

functional                 19706
non functional             18885
functional needs repair     3048
Name: status_group, dtype: int64

In [66]:
df['date_recorded']

0        2011-03-14
1        2013-03-06
2        2013-02-25
3        2013-01-28
4        2011-07-13
            ...    
59395    2013-05-03
59396    2011-05-07
59397    2011-04-11
59398    2011-03-08
59399    2011-03-23
Name: date_recorded, Length: 59400, dtype: object

In [67]:
sorted(df['date_recorded'].tolist(), reverse = True )
#2002 to 2013

['2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-

'date_recorded' is stored in yy-mm-dd format. We thought about using this column along with 'construction_year' to determine pump age. However, 'construction_year' has too many 0 values for us to construct a meaningful age feature.

In [68]:
df['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [69]:
#altitude of the well
df['gps_height']

0        1390
1        1399
2         686
3         263
4           0
         ... 
59395    1210
59396    1212
59397       0
59398       0
59399     191
Name: gps_height, Length: 59400, dtype: int64

In [70]:
df['longitude'].value_counts()

0.000000     1812
37.540901       2
33.010510       2
39.093484       2
32.972719       2
             ... 
37.579803       1
33.196490       1
34.017119       1
33.788326       1
30.163579       1
Name: longitude, Length: 57516, dtype: int64

In [71]:
#remove filler values in longitude and latitude 
# dropping longitude of 0
df = df.loc[df["longitude"] != 0]
df.longitude.value_counts()

33.090347    2
32.982698    2
37.297680    2
33.010510    2
39.093484    2
            ..
37.579803    1
33.196490    1
34.017119    1
33.788326    1
35.005922    1
Name: longitude, Length: 57515, dtype: int64

In [72]:
df['latitude'].value_counts()

-2.496459    2
-6.964258    2
-6.981884    2
-7.175174    2
-7.104625    2
            ..
-5.726001    1
-9.646831    1
-8.124530    1
-2.535985    1
-2.598965    1
Name: latitude, Length: 57516, dtype: int64

Both latitude and longitude seem to have placeholder values. These values are 0.000000 for 'longitude' and -2.000000e-08 for 'latitude'. There are 1,812 of these placeholders. 

In [73]:
#name of the waterpoint 
df['wpt_name']

0                        none
1                    Zahanati
2                 Kwa Mahundi
3        Zahanati Ya Nanyumbu
4                     Shuleni
                 ...         
59395     Area Three Namba 27
59396       Kwa Yahona Kuvala
59397                 Mashine
59398                  Mshoro
59399         Kwa Mzee Lugawa
Name: wpt_name, Length: 57588, dtype: object

In [74]:
df['wpt_name'].value_counts()

none                3492
Shuleni             1734
Zahanati             814
Msikitini            533
Kanisani             322
                    ... 
Kwa Kadungu            1
Nyanguruguru           1
Kwa Piguli             1
Kalakabanga            1
Kwa Mtunga Lyimo       1
Name: wpt_name, Length: 36720, dtype: int64

We thought 'wpt_name' is probably not a useful feature for classification, similar to 'scheme_name'. The many unique values would limit out model's performance with 37,400 unique values in this column. Some of these are subvillage names (like Shuleni), explaning why some values have much higher value counts than others.

In [75]:
#no context
df['num_private'].value_counts()

0      56831
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

There was no context available for 'num_private', not even from the data source. We decided this was another column that would go unused in modeling since we were unable to identify its meaning. Additionally, the large value counts imbalance was worrying without any business understanding of these values.

In [76]:
#Geographic water basin
df['basin'].value_counts()

Pangani                    8940
Lake Victoria              8535
Rufiji                     7976
Internal                   7785
Lake Tanganyika            6333
Wami / Ruvu                5987
Lake Nyasa                 5085
Ruvuma / Southern Coast    4493
Lake Rukwa                 2454
Name: basin, dtype: int64

In [77]:
#Geographic location
df['subvillage'].value_counts()

Majengo                 494
Shuleni                 492
Madukani                435
Unknown                 371
Kati                    366
                       ... 
Igodimwa                  1
Jengemwanama              1
Buyoga A                  1
Ilolangulu Busenda B      1
Bombambili 2              1
Name: subvillage, Length: 18568, dtype: int64

In [78]:
#geographic location
df['region'].value_counts()

Iringa           5294
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Shinyanga        3977
Arusha           3350
Kagera           3316
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Mwanza           2295
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

Both 'region_code' and 'district_code' are integer types. However, as codes, it's unlikely that there is any meaning to the order or progression of values. Therefore, we decided to treat them as categorical variables and converted them to strings.

In [79]:
#geographic location (coded)
df['region_code'].value_counts()

11    5297
12    4639
3     4379
5     4040
17    3954
18    3324
2     3024
16    2816
10    2640
4     2513
19    2295
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [80]:
#region code is type int, but best presented as a categorical feature
#convert to type string
df['region_code'] = df['region_code'].astype(str)

In [81]:
df['region_code']

0        11
1        20
2        21
3        90
4        18
         ..
59395     3
59396    11
59397    12
59398     1
59399     5
Name: region_code, Length: 57588, dtype: object

In [82]:
#geographic location (coded)
df['district_code'].value_counts()

1     11146
2     10909
3      9998
4      8996
5      4356
6      3586
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

In [83]:
#district code is type int, but best presented as a categorical feature
#convert to type string
df['district_code'] = df['district_code'].astype(str)

In [84]:
df['district_code']

0         5
1         2
2         4
3        63
4         1
         ..
59395     5
59396     4
59397     7
59398     4
59399     2
Name: district_code, Length: 57588, dtype: object

Based on some research and information lookup, 'lga' seems to refer to cities or areas of cities.

In [85]:
#geographic location (city?)
df['lga'].value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Rungwe          1106
Kilosa          1094
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 124, dtype: int64

In [86]:
#geographic location (ward?)
df['ward'].value_counts()

Igosi            307
Imalinyi         252
Siha Kati        232
Mdandu           231
Nduruma          217
                ... 
Nsemulwa           1
Ifinga             1
Kihangimahuka      1
Themi              1
Mawenzi            1
Name: ward, Length: 2033, dtype: int64

The 'population' column has a large number of '0' values for the population around the pump.

In [87]:
#population around the well
df['population'].value_counts()

0       19569
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [88]:
df['public_meeting'].value_counts()

True     52713
False     4875
Name: public_meeting, dtype: int64

The 'recorded_by' column told us that the data was collected by a group called GeoData Consultants Ltd. This fact was useful for our data understanding but inessential to our modeling.

In [89]:
#feature unimportant for modeling
df['recorded_by'].value_counts()

GeoData Consultants Ltd    57588
Name: recorded_by, dtype: int64

As mentioned earlier, there were numerous 0 values in 'construction_year'. 20,709 values represents about one-third of our total amount of data.

In [90]:
#how do we treat 0's in construction year?
#age as an ordinal encoded variable to properly treat 0's?
df['construction_year'].value_counts()

0       18897
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [91]:
df[df['construction_year'] != 0]['construction_year'].describe()
#contruction years range from 1960 to 2013, with 20709 values of 0

count    38691.000000
mean      1996.814686
std         12.472045
min       1960.000000
25%       1987.000000
50%       2000.000000
75%       2008.000000
max       2013.000000
Name: construction_year, dtype: float64

From the known construction years, we saw a range from 1960 to 2013. This gave us an idea of the vastness of this data set.

### Similar Columns

The following section identifies groups of columns that all contain similar information. Within these groups, some columns are more granular than others. For modeling, we only wanted to use a single column for each group so that we wouldn't have multiple features all encapsulating essentially the same information.

In [92]:
df['extraction_type'].value_counts()

gravity                      26696
nira/tanira                   7361
other                         6160
submersible                   4688
swn 80                        3448
mono                          2817
india mark ii                 2284
afridev                       1659
ksb                           1358
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  91
cemo                            90
other - play pump               85
climax                          32
walimi                          20
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [93]:
df['extraction_type_group'].value_counts()

gravity            26696
nira/tanira         7361
other               6160
submersible         6046
swn 80              3448
mono                2817
india mark ii       2284
afridev             1659
rope pump            451
other handpump       336
other motorpump      122
wind-powered         117
india mark iii        91
Name: extraction_type_group, dtype: int64

In [94]:
df['extraction_type_class'].value_counts()

gravity         26696
handpump        15179
other            6160
submersible      6046
motorpump        2939
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

'extraction_type' is the most granular, and 'extraction_type_class' is the most broad.

In [95]:
#how does this differ from scheme_management
df['management'].value_counts()

vwc                 39746
wug                  5556
water board          2932
wua                  2533
private operator     1970
parastatal           1696
water authority       902
other                 840
company               685
unknown               551
other - school         99
trust                  78
Name: management, dtype: int64

In [96]:
df['scheme_management'].value_counts()

VWC                 36143
WUG                  4249
None                 3751
Water authority      3151
WUA                  2882
Water Board          2747
Parastatal           1607
Private operator     1063
Company              1061
Other                 765
SWC                    97
Trust                  72
Name: scheme_management, dtype: int64

In [97]:
df['management_group'].value_counts()

user-group    50767
commercial     3635
parastatal     1696
other           939
unknown         551
Name: management_group, dtype: int64

'management' is the most granular, and 'management_group' is the most broad. Capitalization was also inconsistent between columns.

In [98]:
df['payment'].value_counts()

never pay                24380
pay per bucket            8953
pay monthly               8229
unknown                   7654
pay when scheme fails     3843
pay annually              3626
other                      903
Name: payment, dtype: int64

In [99]:
df['payment_type'].value_counts()

never pay     24380
per bucket     8953
monthly        8229
unknown        7654
on failure     3843
annually       3626
other           903
Name: payment_type, dtype: int64

The value counts for 'payment' and 'payment_type' aligned perfectly. They relayed the same information with differently named categories.

In [100]:
df['water_quality'].value_counts()

soft                  49431
salty                  4772
unknown                1661
milky                   803
coloured                479
salty abandoned         228
fluoride                199
fluoride abandoned       15
Name: water_quality, dtype: int64

In [101]:
df['quality_group'].value_counts()

good        49431
salty        5000
unknown      1661
milky         803
colored       479
fluoride      214
Name: quality_group, dtype: int64

'water_quality' and 'quality_group' convey the same information with some slight differences in values. water_quality is more granular.

In [102]:
df['quantity'].value_counts()

enough          32260
insufficient    14564
dry              5990
seasonal         4001
unknown           773
Name: quantity, dtype: int64

In [103]:
df['quantity_group'].value_counts()

enough          32260
insufficient    14564
dry              5990
seasonal         4001
unknown           773
Name: quantity_group, dtype: int64

'quantity' and 'quantity_group' had identical value counts and category names.

In [104]:
df['source'].value_counts()

spring                  17006
shallow well            15499
machine dbh             10826
river                    9612
rainwater harvesting     2218
hand dtw                  873
dam                       649
lake                      639
other                     202
unknown                    64
Name: source, dtype: int64

In [105]:
df['source_type'].value_counts()

spring                  17006
shallow well            15499
borehole                11699
river/lake              10251
rainwater harvesting     2218
dam                       649
other                     266
Name: source_type, dtype: int64

In [106]:
df['source_class'].value_counts()

groundwater    44204
surface        13118
unknown          266
Name: source_class, dtype: int64

'source' is the most granular, and 'source_class' is the most broad.

In [107]:
df['waterpoint_type'].value_counts()

communal standpipe             28375
hand pump                      16181
other                           6167
communal standpipe multiple     5959
improved spring                  783
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [108]:
df['waterpoint_type_group'].value_counts()

communal standpipe    34334
hand pump             16181
other                  6167
improved spring         783
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

waterpoint_type and waterpoint_type_group are almost identical except the communal standpipe multiple group in 'waterpoint_type' is included in 'communal standpipe' in 'waterpoint_type_group'

In [109]:
df['id'].value_counts()

2047     1
20959    1
4759     1
661      1
2708     1
        ..
62836    1
52595    1
50546    1
56689    1
0        1
Name: id, Length: 57588, dtype: int64

In [110]:
df['id_label'].value_counts()

2047     1
20959    1
4759     1
661      1
2708     1
        ..
62836    1
52595    1
50546    1
56689    1
0        1
Name: id_label, Length: 57588, dtype: int64

We confirmed that no duplicate 'id' appeared in the dataset

In [111]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


In [112]:
#FINAL NULLS CHECK
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

Our cleaning removed all nulls in the dataset.

After exploring the columns, we adjusted some values that appeared as placeholders. We also identified columns that continued similar or identical information and determined how we would only implement one column from every similar group into our model.

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57588 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     57588 non-null  int64  
 1   amount_tsh             57588 non-null  float64
 2   date_recorded          57588 non-null  object 
 3   funder                 57588 non-null  object 
 4   gps_height             57588 non-null  int64  
 5   installer              57588 non-null  object 
 6   longitude              57588 non-null  float64
 7   latitude               57588 non-null  float64
 8   wpt_name               57588 non-null  object 
 9   num_private            57588 non-null  int64  
 10  basin                  57588 non-null  object 
 11  subvillage             57588 non-null  object 
 12  region                 57588 non-null  object 
 13  region_code            57588 non-null  object 
 14  district_code          57588 non-null  object 
 15  lg

In [114]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


We saved the dataframe containing all fo the cleaned data and the 'target_group' into a separate CSV file available in the data folder of this project's repository. This file was imported into our modeling notebook.

In [115]:
#SAVE CLEAN DATA OFF TO CSV FILE FOR IMPORT INTO OTHER NOTEBOOKS
#index= False
#df.to_csv('./data/water_well_train_clean.csv', index_label=False)

Please proceed to the modeling notebook.